In [46]:
## This file is used to split data into series of arrays and their corresponding MI task.
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as la
file = sio.loadmat('dataset/CLA/CLA-SubjectJ-170510-3St-LRHand-Inter.mat') #replace with .mat file name
header=file['__header__']
version=file['__version__']
glob=file['__globals__']
ans=file['ans']


x=file['x']
o=file['o'][0][0]
data=o['data']
data = np.transpose(data)
data = data[0:21,:]
print(data)
nS=o['nS'][0][0]
#values of structure seem to be 2D numpy arrays, if originally a scalar in Matlab.
#use [0][0] to get scalar.
print("Number of samples: {numSamples}".format(numSamples=nS))
test=o['id'][0] #id value became a 1D array of size 1 for some reason. use [0] to get value
print("Dataset ID: {id}".format(id=test))
chnames=o['chnames'][:,0] #[:,0] converts from 2D array back to 1D array
print("Channel names: {channelNames}".format(channelNames=chnames))
markers = o['marker']
## The markers are all still individual arrays of size 1x1, so we convert them to an array with single values
markersArray = []
for marker in markers:
    markersArray.append(marker[0])
markersArray = np.asarray(markersArray)
#For this dataset, the markers are 0, 1, or 2.
# 1 - Left Hand MI, 2 - Right Hand MI, 3 - Passive State, 0 - Rest (???)

[[ 13.2 -21.6 -13.2 ... -39.6 -40.2 -24.6]
 [  6.6  -5.4   8.4 ... -10.8 -14.4  -6.6]
 [ -1.2   2.4   0.6 ...  -5.4  -3.6 -12. ]
 ...
 [  5.4  -8.4   6.  ... -10.8  -5.4  -6.6]
 [ -0.6 -12.   -3.6 ...  -3.6  -1.2  -0. ]
 [  4.8  -9.6  -0.6 ...   4.2   2.4   1.2]]
Number of samples: 621884
Dataset ID: 201705101500.32BEA9DD
Channel names: [array(['Fp1'], dtype='<U3') array(['Fp2'], dtype='<U3')
 array(['F3'], dtype='<U2') array(['F4'], dtype='<U2')
 array(['C3'], dtype='<U2') array(['C4'], dtype='<U2')
 array(['P3'], dtype='<U2') array(['P4'], dtype='<U2')
 array(['O1'], dtype='<U2') array(['O2'], dtype='<U2')
 array(['A1'], dtype='<U2') array(['A2'], dtype='<U2')
 array(['F7'], dtype='<U2') array(['F8'], dtype='<U2')
 array(['T3'], dtype='<U2') array(['T4'], dtype='<U2')
 array(['T5'], dtype='<U2') array(['T6'], dtype='<U2')
 array(['Fz'], dtype='<U2') array(['Cz'], dtype='<U2')
 array(['Pz'], dtype='<U2')]


In [47]:
## Find the starting indeces where the marker changes
changeIdxs = np.where(np.transpose(markersArray)[:-1] != np.transpose(markersArray)[1:])[0]
print("Number of index changes: {idxChanges}".format(idxChanges=changeIdxs.shape[0]))
## Split the data so that it has its matching marker
dataSplit = np.array_split(data, changeIdxs[:-1], axis=1)
splitCount = 0
for splitData in dataSplit:
    splitCount += 1
print("Number of arrays in data split: {num}".format(num=splitCount))
## Retrieve the marker values for each of the change indeces (changeIdxs)
markerTargets = markersArray[changeIdxs];
print("Number of marker targets: {numTargets}".format(numTargets=markerTargets.shape[0]))

Number of index changes: 1800
Number of arrays in data split: 1800
Number of marker targets: 1800


In [48]:
## To Apply CSP, we first only get the indeces for MI tasks 1 and 2 (left and right hand, respectively.)
LeftIdxs = np.where(markerTargets == 1)
RightIdxs = np.where(markerTargets == 2)
NoneIdxs = np.where(markerTargets == 3)
numLeftIdx = LeftIdxs[0].shape

In [49]:
leftData = [];
for leftIndex in LeftIdxs[0]:
    #print(leftIndex)
    #print("Dimensions of index: {ind}".format(ind=dataSplit[leftIndex].shape))
    if(dataSplit[leftIndex].shape[1] != 170):
        continue
    else:
        leftData.append(np.transpose(dataSplit[leftIndex]))
leftData = np.asarray(leftData)
leftData.shape

(288, 170, 21)

In [50]:
rightData = [];
for rightIndex in RightIdxs[0]:
    #print(leftIndex)
    #print("Dimensions of index: {ind}".format(ind=dataSplit[leftIndex].shape))
    if(dataSplit[rightIndex].shape[1] != 170):
        continue
    else:
        rightData.append(np.transpose(dataSplit[rightIndex]))
rightData = np.asarray(rightData)
rightData.shape

(327, 170, 21)

In [51]:
noneData=[]
for noneIndex in NoneIdxs[0]:
    if(dataSplit[noneIndex].shape[1] != 170):
        continue
    else:
        noneData.append(np.transpose(dataSplit[noneIndex]))
noneData=np.asarray(noneData)
noneData.shape

(284, 170, 21)

In [52]:
## Only keep the top 288 samples, so that left and right data are equal #why start at index 1?
keep=284;
rightDataSub = rightData[1:(keep+1)]
leftDataSub = leftData[1:(keep+1)]

In [53]:
#Construct the target array and merge the data
leftTargets = np.tile(np.array([0]),(keep,1))
rightTargets = np.tile(np.array([2]), (keep,1))
noneTargets = np.tile(np.array([1]), (keep,1))
markerTargets = np.vstack((leftTargets, noneTargets, rightTargets))
lrData = np.vstack((leftDataSub, noneData, rightDataSub))

#Sanity Check
print("lrData Shape: {arg1}\tmarkerTargets Shape: {arg2}".format(arg1=lrData.shape, arg2=markerTargets.shape))

lrData Shape: (852, 170, 21)	markerTargets Shape: (852, 1)


In [54]:
## Construct LSTM using Tensorflow + Keras
# Import Libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras import optimizers

In [55]:
## Shuffle the data
lrData, markerTargets = shuffle(lrData, markerTargets, random_state=0)

In [56]:
## Split into train and test sets
lrDataTrain, lrDataTest, markerTargetsTrain, markerTargetsTest = train_test_split(lrData, markerTargets, test_size=0.3, random_state=1)
markerTargetsTrain.shape

(596, 1)

In [57]:
## Construct LSTM using Tensorflow + Keras
# Import Libraries
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [58]:
#reshape to be compatible with SVM
lrData = lrData.reshape(lrData.shape[0], -1) #reshape so that each row is an action. And contains all the channels and voltages
markerTargets = markerTargets.reshape(-1) #reshape to vector

In [59]:
## Shuffle the data
lrData, markerTargets = shuffle(lrData, markerTargets, random_state=0)

## Split into train and test sets
lrDataTrain, lrDataTest, markerTargetsTrain, markerTargetsTest = train_test_split(lrData, markerTargets, test_size=0.3, random_state=1)

# SVM Classifier
clf = SVC(kernel = 'rbf', random_state = 42)
clf.fit(lrDataTrain, markerTargetsTrain)

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=42,
    shrinking=True, tol=0.001, verbose=False)

In [60]:
trainPerc=clf.score(lrDataTrain, markerTargetsTrain)
testPerc=clf.score(lrDataTest, markerTargetsTest)

print("#################################")
print("#################################")
print("Training Performance:\nPercent Accuracy: {:.3f}%".format(trainPerc*100))
print("#################################")
print("#################################")
print("Testing Performance:\nPercent Accuracy: {:.3f}%".format(testPerc*100))
print("#################################")
print("#################################")

#################################
#################################
Training Performance:
Percent Accuracy: 100.000%
#################################
#################################
Testing Performance:
Percent Accuracy: 31.641%
#################################
#################################


In [61]:
cm = confusion_matrix(markerTargetsTest, clf.predict(lrDataTest))
print(cm)
print("Accuracy score of liking  ")
print(accuracy_score(markerTargetsTest, clf.predict(lrDataTest))*100)

[[81  0  0]
 [82  0  0]
 [93  0  0]]
Accuracy score of liking  
31.640625


In [62]:
print(markerTargetsTrain)

print(clf.predict(lrDataTrain))

[2 2 2 0 1 2 1 0 1 0 0 2 0 0 0 0 2 2 1 1 0 2 1 0 1 2 1 2 1 1 1 0 0 0 0 1 1
 1 1 1 0 2 1 1 1 1 1 2 1 0 1 2 0 1 2 0 2 0 1 0 2 0 0 1 2 0 0 2 2 1 1 0 2 0
 0 2 1 1 1 1 2 0 0 0 2 0 2 2 0 0 2 0 2 0 0 1 0 0 1 0 2 1 1 0 0 0 1 1 1 0 2
 0 0 1 1 0 0 0 0 2 2 0 0 0 2 2 0 2 2 1 1 2 0 0 0 1 1 0 2 1 0 1 0 0 1 0 2 1
 1 1 0 0 2 0 1 2 0 2 1 2 1 2 1 1 1 1 1 2 2 2 0 1 0 1 1 2 1 1 1 2 0 1 0 1 2
 2 0 1 2 1 2 2 0 1 1 2 2 2 1 2 1 0 0 2 0 1 1 1 0 2 2 2 2 1 0 2 0 2 0 1 2 0
 0 0 1 1 0 1 1 1 0 2 2 1 0 2 2 0 2 0 1 2 2 1 1 0 1 0 2 0 1 2 2 1 1 1 1 1 1
 2 2 0 2 0 2 2 0 1 1 2 0 2 1 0 0 0 0 1 1 1 2 0 0 0 2 0 0 2 0 1 1 2 1 2 0 2
 2 2 2 2 0 2 1 2 1 0 1 1 0 1 1 0 0 0 2 2 2 0 2 1 1 1 0 2 0 1 0 2 1 1 1 2 1
 2 2 0 1 1 2 0 0 1 2 0 2 0 0 0 0 2 0 2 2 0 1 2 2 0 1 2 0 0 1 0 0 2 0 1 1 0
 0 1 2 0 1 0 2 1 1 2 2 0 1 1 1 1 2 2 2 2 2 0 1 2 1 1 0 0 0 2 2 1 2 1 2 0 2
 2 1 1 0 2 1 0 0 1 1 2 0 0 2 1 2 0 1 1 1 2 2 1 0 1 1 2 0 1 2 1 2 0 2 1 0 1
 2 0 2 0 1 1 0 0 0 2 2 1 1 0 1 0 0 0 2 2 2 2 1 0 0 0 2 1 2 0 0 1 2 1 2 1 1
 1 0 1 2 0 0 2 2 1 0 0 0 

In [63]:
print(clf.predict(lrDataTest))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
